In [2]:
import tensorflow as tf
def generate_fgsm(image, model, epsilon):
    # Ensure the image has the correct shape (e.g., [1, 32, 32, 3] for CIFAR-10)
    image = tf.convert_to_tensor(image)
    image = tf.reshape(image, (1, *image.shape))

    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        target_class = tf.argmax(prediction[0])
        loss = tf.keras.losses.sparse_categorical_crossentropy(target_class, prediction)

    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    perturbed_image = image + epsilon * signed_grad
    perturbed_image = tf.clip_by_value(perturbed_image, 0, 1)  # Ensure pixel values are in [0, 1] range

    return perturbed_image.numpy()[0]

# Test on image from numbers dataset

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load the FCN model you created earlier
fcn_model = tf.keras.models.load_model("mnist_model.h5")

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(_, _), (test_images, test_labels) = mnist.load_data()
test_images = test_images / 255.0  # Preprocess the data

# Select a random image from the test dataset
random_index = np.random.randint(0, len(test_images))
input_image = test_images[random_index]
original_label = test_labels[random_index]

# Define an epsilon value for the FGSM attack
epsilon = 0.1

# Use the generate_fgsm function to generate a perturbed image
perturbed_image = generate_fgsm(input_image, fcn_model, epsilon)

# Calculate the difference between the original and perturbed images
difference_image = input_image - perturbed_image

# Perform inference on the perturbed image
perturbed_image = perturbed_image[np.newaxis, ...]  # Add a batch dimension
predictions = fcn_model.predict(perturbed_image)

# Get the predicted class label
predicted_label = np.argmax(predictions)
confidence = np.max(predictions)

# Display the original image, perturbed image, and the difference
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(input_image, cmap='gray')
plt.title(f'Original Label: {original_label}')
plt.axis('off')

plt.subplot(132)
plt.imshow(perturbed_image[0, :, :, 0], cmap='gray')
plt.title(f'Predicted Label: {predicted_label}\n(with confidence: {confidence:.2f})')
plt.axis('off')

plt.subplot(133)
plt.imshow(difference_image[0, :, :, 0], cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Difference Image')
plt.axis('off')

plt.show()

4